In [214]:
from collections import deque
import time

class AhoCorasick(object):
    def __init__(self, dictionary, values):
        t = time.time()
        self.trie, self.outputs, self.parents = self.build_trie(dictionary, values)
        td = time.time() - t
        print('build_trie took', td)
        t = time.time()
        self.build_fails()
        td = time.time() - t
        print('build_fails took', td)
        t = time.time()
        self.memoize_dict()
        td = time.time() - t
        print('memoize_dict took', td)

    def build_trie(self, dictionary, values):
        trie = [{}]
        outputs = {}
        parents = {}
        state = 0
        for index, word in enumerate(dictionary):
            state = 0
            for char in word:
                if char not in trie[state]:
                    new = len(trie)
                    trie[state][char] = new
                    parents[new] = (state, char)
                    trie.append({})
                state = trie[state][char]
            if state in outputs:
                outputs[state] += values[index]
            else:
                outputs[state] = values[index]
        return trie, outputs, parents
    
    def build_fails(self):
        self.fails = { state: 0 for state in self.trie[0].values() }
        states = deque(self.trie[0].values())  # BFS
        while states:
            state = states.popleft()
            parent, char = self.parents[state]
            for parent_suffix in self.suffixes(parent):
                # Follow fails until one has a child with same letter
                suffix = self.trie[parent_suffix].get(char)
                if suffix:
                    self.fails[state] = suffix
                    break
            if state not in self.fails:
                self.fails[state] = 0  # Default to fail to root
            for next_state in self.trie[state].values():
                states.append(next_state)

    def memoize_dict(self):
        self.memoize_output = {}
        states = deque(self.trie[0].values())
        while states:
            state = states.popleft()
            for suffix in self.suffixes(state):
                if suffix in self.outputs:
                    if state in self.memoize_output:
                        self.memoize_output[state].append(suffix)
                    else:
                        self.memoize_output[state] = [suffix]
            for next_state in self.trie[state].values():
                states.append(next_state)
    
    def suffixes(self, state):
        suffix = self.fails.get(state)
        while suffix is not None:
            yield suffix
            suffix = self.fails.get(suffix)
    
    def goto(self, state, letter):
        # try child
        next_state = self.trie[state].get(letter)
        if next_state:
            return next_state
        # try suffix children
        for suffix in self.suffixes(state):
            next_state = self.trie[suffix].get(letter)
            if next_state:
                return next_state
        return 0  # if all fails return root

    def value(self, string):
        t = time.time()
        state = 0
        val = 0
        for c in string:
            state = self.goto(state, c)
            val += self.outputs.get(state) or 0
            if state in self.memoize_output:
                val += sum([self.outputs[s] for s in self.memoize_output.get(state)])
        td = time.time() - t
        print('value took', td)
        return val

aho = AhoCorasick(['a', 'ab', 'bab', 'bc', 'bca', 'c', 'caa'], [1, 1, 1, 1, 1, 1, 1])
assert aho.fails == {1: 0, 3: 0, 8: 0, 2: 3, 4: 1, 6: 8, 9: 1, 5: 2, 7: 9, 10: 1}
assert aho.memoize_output == {4: [1], 6: [8], 9: [1], 5: [2], 7: [1], 10: [1]}
print(aho.value('abccab'))

aho = AhoCorasick(['he', 'she', 'his', 'hers'], [1, 1, 1, 1])
assert aho.fails == {1: 0, 3: 0, 2: 0, 6: 0, 4: 1, 8: 0, 7: 3, 5: 2, 9: 3}
print(aho.value('ahishers'))

build_trie took 1.4781951904296875e-05
build_fails took 2.384185791015625e-05
memoize_dict took 2.2172927856445312e-05
value took 1.6689300537109375e-05
7
build_trie took 2.8133392333984375e-05
build_fails took 2.2172927856445312e-05
memoize_dict took 1.7881393432617188e-05
value took 1.1205673217773438e-05
4


In [61]:
def counts(gene, dna):
    for i in range(len(dna)):
        yield dna[i:i+len(gene)] == gene

def dna_health(health, genes, first, last, dna):
    return sum([
        sum(counts(genes[i], dna)) * health[i]
        for i in range(first, last+1)
    ])

In [31]:
class Node(object):
    def __init__(self, path, character, parent, suffix=None):
        self.path = path
        self.character = character
        self.parent = parent
        self.suffix = suffix
        self.health = None
    
    def __repr__(self):
        return '<Node character:{}, parent:{}, suffix:{} >'.format(
            self.character,
            '--' if not self.parent else '<Node path:'+self.parent.path+'>',
            '--' if not self.suffix else '<Node path:'+self.suffix.path+'>'
        )

In [72]:
class Node(object):
    def __init__(self, path, value, parent):
        self.path = path
        self.children = {}
        self.value = value
        self.parent = parent
        self.suffix = None
        self.dict_suffix = None

    def __getitem__(self, letter):
        return self.children.get(letter)
    
    def __contains__(self, letter):
        return letter in self.children
    
    def __setitem__(self, letter, node):
        self.children[letter] = node
    
    def __repr__(self):
        return '{}\t\t{}\t\t{}\t\t{}\n'.format(
            "'"+self.path+"'",
            self.value,
            '--' if not self.suffix else "'"+self.suffix.path+"'",
            '--' if not self.dict_suffix else "'"+self.dict_suffix.path+"'"
        ) + ''.join([
            child.__repr__() 
            for child in self.children.values()
        ])


class Trie(object):
    def __init__(self, dictionary, values):
        self.trie_root = Node('', 0, None)

        for index, gene in enumerate(dictionary):
            curr_node = self.trie_root
            for i, letter in enumerate(gene):
                in_dict = i == len(gene) - 1  # full gene
                node_value = values[index] if in_dict else 0
                if letter not in curr_node:
                    in_dict = i == len(gene) - 1
                    curr_node[letter] = Node(gene[:i+1], node_value, curr_node)
                else:
                    curr_node[letter].value += node_value
                curr_node = curr_node[letter]
        
        self._put_suffixes(self.trie_root, '')
        self._put_dict_suffixes(self.trie_root)

    def value(self, haystack):
        total = 0
        node = self.trie_root
        for straw in haystack:
            node = self._next_good_node(node, straw)
            if node.value:
                total += node.value
            total += sum(self._value_dict_suffixes(node))
        return total
    
    def _value_dict_suffixes(self, node):
        suffix = node.dict_suffix
        while suffix:
            yield suffix.value
            suffix = suffix.dict_suffix
    
    def _next_good_node(self, node, letter):
        if node[letter]:
            return node[letter]
        if not node.suffix:
            return self.trie_root[letter] if self.trie_root[letter] else self.trie_root
        cursor = node.suffix
        while cursor != self.trie_root and not cursor[letter]:
            cursor = cursor.suffix if cursor.suffix else self.trie_root
        return cursor[letter] if cursor[letter] else self.trie_root

    def _put_suffixes(self, node, letter):
        cursor = node.parent
        while cursor:
            suffix = cursor[letter]
            if suffix and suffix != node:
                node.suffix = suffix
                break
            elif cursor == self.trie_root:
                node.suffix = self.trie_root
                break
            cursor = cursor.suffix
        for key, child in node.children.items():
            self._put_suffixes(child, key)
    
    def _put_dict_suffixes(self, node):
        cursor = node.suffix
        while cursor:
            if cursor.value:
                node.dict_suffix = cursor
            cursor = cursor.suffix
        for child in node.children.values():
            self._put_dict_suffixes(child)

    def __repr__(self):
       return self.trie_root.__repr__()

trie = Trie(['a', 'ab', 'bab', 'bc', 'bca', 'c', 'caa'], [1, 1, 1, 1, 1, 1, 1])
trie.value('abccab')

trie = Trie(['hlimdfbfnv', 'mguznrcpfc', 'nrmweeookb', 'drolersfwh', 'ckpykeqotx', 'sioefulviv', 'wnmkasbuzz', 
             'ddkscwwukr', 'rfzhjgbwbl', 'rzagjaymua', 'mxdyrhunbg', 'eulfdxogtr', 'rnrrtctrpp', 'tdmzbfgxsi', 
             'fyereiquol', 'liyebrhvly', 'kgbzfeembz', 'wgxazdirzx', 'flfdrgxydi', 'woqzpwdvkg', 'ugpuiqxrix', 
             'qnkxsbfpcj', 'zazqaqmdly', 'mgebaorzfz', 'yxoiuhmayo', 'lyqkoqacwn', 'aivgjxucxc', 'cxzsgwbuya', 
             'klyavotxsp', 'muzickfwmc', 'aqccjiakey', 'mojmqgajfu', 'yrozzqjfpw', 'jrmltxvtkz', 'twpejgmlpr', 
             'gqlwpknbre', 'xdvlqplmkv', 'ngtfmelzsc', 'qyudukojnh', 'nkmjxdairm', 'fgublhhygz', 'byxvcuhsdu', 
             'btgocgreqk', 'syqnzeuicc', 'ifahdebmwh', 'jaapoexhio', 'rcmjpnnlxq', 'nfvonauqnt', 'xwtznjdlqn', 
             'bjqnshcgtz', 'yghvwuwrml', 'kmhdlumrhe', 'einwxhebpx', 'bnfilcejts', 'ufebiqxwjh', 'cnprmnysoq', 
             'rrfwbqahzv', 'atagwkwwif', 'dkvsbjhcby', 'surxqvqter', 'oenpljzjhi', 'rkuofwxoaa', 'osugrmdjfh', 
             'bwoolbzmkh', 'wdtrrypqpp', 'qdjmlcbomi', 'wpekdpleex', 'nabhtuhinw', 'zfcksnntcb', 'dyqiktzxzd', 
             'ungxuzubkh', 'almcwgrlbt', 'mftcndxoaw', 'sxjawdzshl', 'zjxonvwegy', 'ysfruuxtiz', 'payzavecpn', 
             'ppwofjjbop', 'bojghfaeyj', 'golgpodtst', 'hhifwprhqf', 'xuvgacodjm', 'orcbxrpbnj', 'uwtebrtsyl', 
             'zxfugizuli', 'gzzjawcszp', 'btnwxrnqlm'],
            [1803633, 3357388, 1112112, 5656776, 4438527, 3841975, 4102090, 2113339, 2977711, 1709727, 1666821, 
             4167887, 3742911, 1948785, 3057238, 1940358, 4574138, 4598641, 2922682, 1839758, 4562812, 1508583, 
             2531144, 3192788, 4971388, 3448060, 2579952, 4895338, 5133938, 5667253, 5417655, 3453923, 3024642, 
             5529768, 3110699, 3979521, 5888095, 3729142, 2609212, 2865806, 1955221, 3276034, 3550045, 4698132, 
             1741171, 5607283, 5638490, 2831662, 1722277, 2561172, 3671420, 5285089, 5586108, 5202564, 4994229, 
             2073848, 5166977, 1574181, 5969186, 1817267, 1241435, 2903194, 1787542, 5782429, 4949314, 1414593, 
             1278302, 4837409, 4143735, 2887514, 4219567, 5098956, 2679900, 1769612, 1313440, 5937424, 3893247, 
             3468283, 2769086, 2131876, 5029455, 2956858, 3933318, 2131915, 4675774, 2927547, 3205764])
#print(trie)
trie.value('zjxonvwegyeulfdxogtr')

trie = Trie(['mguznrcpfc', 'nrmweeookb', 'drolersfwh', 'ckpykeqotx', 'sioefulviv', 'wnmkasbuzz', 'ddkscwwukr', 
             'rfzhjgbwbl', 'rzagjaymua', 'mxdyrhunbg', 'eulfdxogtr', 'rnrrtctrpp', 'tdmzbfgxsi', 'fyereiquol', 
             'liyebrhvly', 'kgbzfeembz', 'wgxazdirzx', 'flfdrgxydi', 'woqzpwdvkg', 'ugpuiqxrix', 'qnkxsbfpcj', 
             'zazqaqmdly', 'mgebaorzfz', 'yxoiuhmayo', 'lyqkoqacwn', 'aivgjxucxc', 'cxzsgwbuya', 'klyavotxsp', 
             'muzickfwmc', 'aqccjiakey', 'mojmqgajfu', 'yrozzqjfpw', 'jrmltxvtkz', 'twpejgmlpr', 'gqlwpknbre', 
             'xdvlqplmkv', 'ngtfmelzsc', 'qyudukojnh', 'nkmjxdairm', 'fgublhhygz', 'byxvcuhsdu', 'btgocgreqk', 
             'syqnzeuicc', 'ifahdebmwh', 'jaapoexhio', 'rcmjpnnlxq', 'nfvonauqnt', 'xwtznjdlqn', 'bjqnshcgtz', 
             'yghvwuwrml', 'kmhdlumrhe', 'einwxhebpx', 'bnfilcejts', 'ufebiqxwjh', 'cnprmnysoq', 'rrfwbqahzv', 
             'atagwkwwif', 'dkvsbjhcby', 'surxqvqter', 'oenpljzjhi', 'rkuofwxoaa', 'osugrmdjfh', 'bwoolbzmkh', 
             'wdtrrypqpp'],
            [3357388, 1112112, 5656776, 4438527, 3841975, 4102090, 2113339, 2977711, 1709727, 1666821, 4167887, 
             3742911, 1948785, 3057238, 1940358, 4574138, 4598641, 2922682, 1839758, 4562812, 1508583, 2531144, 
             3192788, 4971388, 3448060, 2579952, 4895338, 5133938, 5667253, 5417655, 3453923, 3024642, 5529768, 
             3110699, 3979521, 5888095, 3729142, 2609212, 2865806, 1955221, 3276034, 3550045, 4698132, 1741171, 
             5607283, 5638490, 2831662, 1722277, 2561172, 3671420, 5285089, 5586108, 5202564, 4994229, 2073848, 
             5166977, 1574181, 5969186, 1817267, 1241435, 2903194, 1787542, 5782429, 4949314])

trie.value('bjqnshcgtzugpuiqxrix')

7123984

In [219]:
with open('input3.txt') as inputs:
    n = int(inputs.readline())
    genes = inputs.readline().rstrip().split()
    health = list(map(int, inputs.readline().rstrip().split()))
    s = int(inputs.readline())

    least = float('inf')
    most = float('-inf')

    for s_itr in range(s):
        firstLastd = inputs.readline().split()

        first = int(firstLastd[0])
        last = int(firstLastd[1])
        dna = firstLastd[2]

        print('building trie from '+str(last-first+1)+' genes')
        aho = AhoCorasick(genes[first:last+1], health[first:last+1])
        print('searching '+str(len(dna))+' long sequence')
        score = aho.value(dna)
        print(score)
        #score_old = dna_health(health, genes, first, last, dna)
        
        #if score != score_old:
        #    print('wrong')

        if score < least:
          least = score
        if score > most:
          most = score
    
    print(least, most)

building trie from 100000 genes
build_trie took 1.3891017436981201
build_fails took 4.066978216171265
memoize_dict took 5.951034784317017
searching 19 long sequence
value took 4.792213439941406e-05
7353994
building trie from 45417 genes
build_trie took 0.6190502643585205
build_fails took 1.8430819511413574
memoize_dict took 2.626586437225342
searching 100 long sequence
value took 0.00020360946655273438
0
building trie from 56216 genes
build_trie took 0.7809844017028809
build_fails took 2.411529779434204
memoize_dict took 3.3719418048858643
searching 100 long sequence
value took 0.00020241737365722656
0
building trie from 22864 genes
build_trie took 0.3155367374420166
build_fails took 0.9200310707092285
memoize_dict took 1.365328311920166
searching 100 long sequence
value took 0.00021767616271972656
0
building trie from 56575 genes
build_trie took 0.8043544292449951
build_fails took 2.4044101238250732
memoize_dict took 3.3535852432250977
searching 100 long sequence
value took 0.00021862

KeyboardInterrupt: 